In [ ]:
 import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!pip install pickle5  
from pickle5 import pickle

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rc('font', family='NanumBarunGothic')

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math


###############################################################
#############################엘보함수##########################
###############################################################

def elbow(data, length):
    sse = [] # sum of squre error 오차제곱합
    for i in range(1, length):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(data)
        # SSE 값 저장
        sse.append(kmeans.inertia_)
    plt.plot(range(1, length), sse, 'bo-')
    plt.title("elbow method")
    plt.xlabel("number of clusters")
    plt.ylabel("SSE")
    plt.show()



###############################################################
###########################실루엣함수##########################
###############################################################

def visualize_silhouette(cluster_lists, X_features): 
        
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

def get_lineage(tree, feature_names):
     left      = tree.tree_.children_left
     right     = tree.tree_.children_right
     threshold = tree.tree_.threshold
     features  = [feature_names[i] for i in tree.tree_.feature]

     # get ids of child nodes
     idx = np.argwhere(left == -1)[:,0]     

     def recurse(left, right, child, lineage=None):          
          if lineage is None:
               lineage = [child]
          if child in left:
               parent = np.where(left == child)[0].item()
               split = 'l'
          else:
               parent = np.where(right == child)[0].item()
               split = 'r'

          lineage.append((parent, split, threshold[parent], features[parent]))

          if parent == 0:
               lineage.reverse()
               return lineage
          else:
               return recurse(left, right, parent, lineage)

     for child in idx:
          for node in recurse(left, right, child):
               print(node)


# 군집위한 데이터셋 불러오기

In [ ]:
################################################################################################################################################
################################################################################################################################################
################################################################################################################################################

#################################### 가능하면 여기서 drop할 컬럼 지우고 가야함 ####################################

path = '/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/df_ml_123_AbsNet_범주4_cv.pkl'

df = pd.read_pickle(path)
# dec_df = df[(df['target']==1)].drop(['target','고객번호','A_고객등급','B_고객등급','C_고객등급','D_고객등급'],axis=1)
dec_df = df[(df['target']==1)]
dec_df =dec_df[['Recency_Net', '객단가_Net', 'Monetary_Net', '구매횟수_Net', '방문별거래횟수_Net',
       'A_amt_Net', 'B_amt_Net', 'C_amt_Net', 'D_amt_Net', '평일_Net', '오전_Net',
       '고관여제품_Net', '사치품_Net', '식품_Net', '저관여제품_Net', '패션_Net', '엥겔지수_Net',
       '연령대', '거주지역랭크', '멤버십_가입개수', '거주지역랭크_범주', '성별_F', '성별_M', '누적구매금액',
       'Recency_cv', '객단가_cv', 'Monetary_cv', '방문별거래횟수_cv', 'A_amt_cv',
       'B_amt_cv', 'C_amt_cv', 'D_amt_cv', '오전_cv', '고관여제품_cv', '사치품_cv',
       '식품_cv', '저관여제품_cv', '패션_cv', '엥겔지수_cv', 'Recency_ma', 'Frequency_ma',
       '내구재_amt_ratio_ma', '준내구재_amt_ratio_ma', '비내구재_amt_ratio_ma',
       '사치품_amt_ratio_ma', '평일_r_ma', 'A_amt_r_ma', 'B_amt_r_ma', 'C_amt_r_ma',
       'D_amt_r_ma', '고객총매출_ma', 'target']]

In [ ]:
dec_df

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
train_df = pd.read_pickle('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/이젠그만하자/4분위_std_cv_test.pkl')

X = train_df.drop(['target','고객번호'],axis=1)
y = train_df['target']

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=42)

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
# dt_pred = lgbm.predict(X_test)

# accuracy_score(y_test, dt_pred)

In [ ]:
from lightgbm import LGBMClassifier

lgbm =LGBMClassifier(n_estimators=400)
evals= [(X_test,y_test)]
lgbm.fit(X_train, y_train)
lgbm_preds = lgbm.predict(X_test)
lgbm_preds_proba = lgbm.predict_proba(X_test)[:,1]

In [ ]:
important_df = pd.Series(lgbm.feature_importances_ , index = X_train.columns)

im_data = important_df.sort_values(ascending=False)[:22].index

In [ ]:
im_data

In [ ]:
def plot_feature_importances_cancer(model):
    plt.figure(figsize=(20,15))
    n_features = X.shape[1]

    plt.barh(range(n_features),model.feature_importances_, align='center')

    plt.yticks(np.arange(n_features),X.columns)

    plt.xlabel("attr importances")

    plt.ylabel("attr")

    plt.ylim(-1,n_features)

plot_feature_importances_cancer(dt)

plt.show()

In [ ]:
df.columns

In [ ]:
important_df = pd.Series(dt.feature_importances_ , index = X_train.columns)

im_data = list(important_df.sort_values(ascending=False)[:22].index)
# im_data.append('거주지역랭크')

In [ ]:
im_data

In [ ]:
# dec_df = X[im_data]
train_df

In [ ]:
train_df.columns

In [ ]:
dec_df = train_df[(train_df['target']==1)][im_data]

dec_df = train_df[(train_df['target']==1)]
dec_df = dec_df[['Recency_Net', '객단가_Net', 'Monetary_Net', '구매횟수_Net',
       '방문별거래횟수_Net', 'A_amt_Net', 'B_amt_Net', 'C_amt_Net',
       '평일_Net', '오전_Net', '고관여제품_Net', '사치품_Net', '식품_Net', '저관여제품_Net',
       '패션_Net', '엥겔지수_Net', '연령대', '거주지역랭크', '멤버십_가입개수', '성별_F',
       '성별_M', '누적구매금액', 'Recency_cv', '객단가_cv', 'Monetary_cv', 'A_amt_cv',
       'B_amt_cv', 'C_amt_cv', '평일_cv', '오전_cv', '고관여제품_cv',
       '사치품_cv', '식품_cv', '저관여제품_cv', '패션_cv', '엥겔지수_cv', 'Recency_ma',
       'Frequency_ma', '고객총매출_ma']]

len(dec_df.columns)

# 데이터셋의 실루엣 및 엘보우 확인

In [ ]:
dec_df

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dec_df[['거주지역랭크_1']] = scaler.fit_transform(dec_df[['거주지역랭크']])
dec_df_c = dec_df.copy()
dec_df_c = dec_df_c.drop(['거주지역랭크'],axis=1)
dec_df_c

In [ ]:
visualize_silhouette([3,4,5,6,7,8,9,10], dec_df_c)

In [ ]:
  elbow(dec_df_c, 13)

In [ ]:
# test_df = df.drop(['target',''])

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,15))

sns.heatmap(dec_df_c.corr(),annot=True, cmap='YlGn')

# 1. 계층적군집 => label값 cluster_df로 통합
# 2. cluster_df에서 군집된 label값 dummie로 one-hot
# 3. one-hot된 label 다시 cluster_df 로 붙이기
# 4. 최종 군집된 데이터 == cluster_df

In [ ]:
from sklearn.cluster import AgglomerativeClustering

kmeans = AgglomerativeClustering(n_clusters=4)
kmeans.fit(dec_df_c)
labels = pd.Series(kmeans.labels_)
dec_df_c = dec_df_c.reset_index(drop=True)

cluster_df = pd.concat([dec_df_c, labels],axis=1)
cluster_df

dum_data = pd.get_dummies(labels)
cluster_df = pd.concat([dec_df_c, dum_data],axis=1)
cluster_df

Decision Tree로 군집 선택지 확인

In [ ]:
cluster_df

In [ ]:
## 함수화해놓은거

## clust_count(군집갯수, 확인하고싶은종속변수, 깊이)

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

def clust_count(군집갯수, 종속변수, depth): # 군집화된 갯수(현재 설정은 4, 확인하고픈 종속변수 이것도 0부터 3, max_depth )

    count = list(np.arange(군집갯수))


    X = cluster_df.drop(count, axis=1)
    y = cluster_df[종속변수]

    X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=42)

    dt = DecisionTreeClassifier(max_depth= depth)
    dt.fit(X_train, y_train)
    pred = dt.predict(X_test)
    acc = accuracy_score(y_test, pred)
    print(acc)

    import matplotlib.pyplot as plt
    # from matplotlib import rc
    from sklearn import tree

    # rc('font',family='Malgun Gothic') # 한글 폰트 설정
    # plt.rcParams['axes.unicode_minus'] = False #

    plt.figure(figsize=(25,10))
    tree.plot_tree(dt,fontsize=400)

    tree.plot_tree(dt, filled=True,
                feature_names = X.columns,
                class_names = 'cluster',
                rounded=True)
    plt.savefig(f'/content/drive/MyDrive/공유문서/1차 프로젝트/5조/이젠그만하자/dt_tree{종속변수}.png')
    plt.show()
    

    return dt

In [ ]:
clust_count(4, 0, 5)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = cluster_df.drop([0,1,2,3], axis=1)
y = cluster_df[3]

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=42)

dt = DecisionTreeClassifier(max_depth=6)
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
acc = accuracy_score(y_test, pred)
print(acc)

import matplotlib.pyplot as plt
# from matplotlib import rc
from sklearn import tree

# rc('font',family='Malgun Gothic') # 한글 폰트 설정
# plt.rcParams['axes.unicode_minus'] = False #

# plt.figure(figsize=(20,15))

# tree.plot_tree(dt, filled=True,
#               feature_names = X.columns,
#               class_names = 'cluster',
#               rounded=True)
# plt.show()

In [ ]:
X

In [ ]:
## 군집 압축해서 보기

from sklearn.tree import export_graphviz
export_graphviz(dt, out_file="tree.dot", class_names=["0", "1","2","3"],
                feature_names=X.columns,
                filled=True)

import graphviz

with open("tree.dot") as f:
    dot_graph = f.read()

display(graphviz.Source(dot_graph))

import pydot
(graph,) = pydot.graph_from_dot_file('tree.dot', encoding='utf8')
graph.write_png('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/이젠그만하자/tree.png')

In [ ]:
import pydot
(graph,) = pydot.graph_from_dot_file('dicisionTree1.dot', encoding='utf8')
graph.write_png('dicisionTree1.png')

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,15))

sns.heatmap(cluster_df.corr(),annot=True, cmap='YlGn')

In [ ]:
## 글로 분류보기

get_lineage(dt, X.columns)

In [ ]:
with open ('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/이젠그만하자/net_df.pkl','wb') as read:
  pickle.dump(cluster_df,read)

In [ ]:
with open ('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/이젠그만하자/net_df.pkl','rb') as read:
  a = pickle.load(read)

In [ ]:
a

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,15))

sns.heatmap(cluster_df.corr(),annot=True, cmap='YlGn')

In [ ]:
## 글로 분류보기

get_lineage(dt, X.columns)

In [ ]:
clust_df.to_pickle('')